# Вариант 4. Васильев Максим, 6373.

## Задача 1.

### Текст задачи 

В цехах $N1$ и $N2$ предприятия производится продукт Y, который в дальнейшем используется в качестве исходного материала для производства изделий в цехе $N3$. Суммарная производительность цехов N1 и N2 зависит от вложения дополнительных средств X. При работе цехов N1 и N2 в течение одного месяца эта зависимость может быть приближённо представлена в виде функций:

- $N1: y = 5 + (x + 40)^{1/3}$
- $N2: y = 7 + (x + 30)^{1/3}$

Функции остатка средств в течении месяца:

- $N1: 0.8x$
- $N2: 0.62x$

Средства выделяемые на оба цеха в течении квартала (3 месяца), составляют 179 единиц; перераспределение производится помесячно. 

Требуется распределить средства на планируемый квартал с целью получения максимального количества продукта Y.

### Формализация задачи

Так-как количество управлений невелико, для решения задачи был выбран метод Динамического программирования.Для решения задачи выбранным методом, нужно выбрать способ описания процесса.

#### Способ описания процесса

1. Этапы   - месяца финансирования, $i = 0..2$
2. Выигрыш - суммарная производительность двух цехов, $y = 12 + (x_1 + 40)^{1/3} + (x_2 + 30)^{1/3}$, где $x_1$ - вложерние в первый цех, $x_2$ - вложение во второй цех
3. Управление - количество средств выделенное на первый цех, т.к. на второй цех будет выделена автоматически оставшаяся часть.
4. Состояние - остаток средств в течении месяца, в первом месяце это $179$

#### Описание в терминах уравнения Беллмана

$S_i$ - состояние на $i$-м этапе  
$u_i$ - управление на $i$-м этапе  
$W_i$ - условный оптимальный выигрыш на всех шагах от i-го и до последнего  
$w_i(S_i, u_i)$ - выигрыш на $i$-м этапе  
$\varphi_i(S_i, u_i)$ - изменение состояния системы на i-м шаге  

Запишем выигрыш и изменение состояния на i-м шаге:  
$w_i(S_i, u_i) = 12 + (u_i + 40)^{1/3} + (S_i - u_i + 30)^{1/3}$  
$\varphi_i(S_i, u_i) = 0.8 u_i + 0.66(S_i - u_i)$  

Тогда основное функциональное уравнение будет иметь следующий вид:

$W_i(S_i) = max(12 + (u_i + 40)^{1/3} + (S_i - u_i + 30)^{1/3} + W_{i+1}(0.66S_i + 0.14u_i))$

In [1]:
# Utility decorators 

def benchmark(func):
    # TODO: Only works right for non-recursive functions
    def decorated_function(*args, **kwargs):
        start = default_timer()
        ret = func(*args, *kwargs)
        end = default_timer()
        print(f"Execution time = {end - start}")
        return ret
    return decorated_function


def cached_function(func):
    cache = {}
    def decorated_function(*args, **kwargs):
        key = tuple(args)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return decorated_function, cache





In [2]:
# Dynamic programming model
# TODO: Construct plan from this
from typing import Callable, Union, Tuple, List


def precision_generator(max_value: float, precision):
    indexes = int(max_value / precision)
    
    def precision_range(value: float):
        max_index = int((value / max_value) * indexes)
        for i in range(max_index):
            yield precision * i
    return precision_range


class DynamicSolver:
    def __init__(self,
                 state_change: Callable[[float, float], float],
                 local_profit_change: Callable[[float, float], float],
                 max_stages: int,
                 max_state: int,
                 precision: float = 0.001):
        self.state_change = state_change
        self.local_profit_change = local_profit_change
        self.max_stages = max_stages
        self.precision_range = precision_generator(max_state, precision)
    
    def global_profit(self, stage: int, state: Union[int, float]) -> Tuple[float, int]:
        if stage >= self.max_stages:
            return (0, None)
        
        profit = [
            (
                self.local_profit_change(state, management) + self.global_profit(stage + 1, self.state_change(state, management))[0],
                management
            ) for management in self.precision_range(state)
        ]
        return max(profit, key=lambda x: x[0])
    
    global_profit, cache = cached_function(global_profit)
    
    def construct_plan(self, stage, state):
        plan = []
        for stage in range(self.max_stages):
            _, management = self.cache[(self, stage, state)]
            plan.append(management)
            state = self.state_change(state, management)
        return plan

    

local_win = lambda s, u: 12 + (u + 40) ** (1/3) + ((s - u) + 30) ** (1/3)
local_state_change = lambda s, u: 0.8 * u + 0.66 * (s - u)

cost_optimizer = DynamicSolver(local_state_change, local_win, 3, 179, 1)

max_y_product = cost_optimizer.global_profit(0, 179)
print(max_y_product)

management_plan = cost_optimizer.construct_plan(0, 179)
print(management_plan)

(64.02965228892725, 111)
[111, 74, 44]


In [3]:
# Management plan check, for clarity
state = 179
profit = 0

for management in management_plan:
    profit += local_win(state, management)
    state = local_state_change(state, management)
    print(f"profit: {profit}, state: {state}")

profit: 21.935510313673433, state: 133.68
profit: 43.26040504422461, state: 98.5888
profit: 64.02965228892727, state: 71.22860800000001


## Задача 2.

### Текст задачи

Цех N3 выпускает продукци в виде трех изделий
TODO

### Формализация задачи

$f(x) = x_{21} + x_{31} + x_{12} + x_{22} + x_{13} + x_{33} -> max$ 

$x_{ij}, i=1..3, j=1..3$ - количество продукции i произведённое цехом j

*Ограничения по количеству продукции:*  
y1: $0.005x_{21} + 0.004x_{31} +  0.003x_{12} + 0.009x_{22} + 0.003x_{13} + 0.005x_{33} <= Y$, где Y - количество продукции полученное в прошлом пункте.  

*Ограничения по времени:*  

y2: $5 x_{21} + 8 x_{31} <= 860$  

y3: $20 x_{12} + 8 x_{22} <= 1500$  

y4: $13 x_{13} + 9 x_{33} <= 870$  

*Ограничения на отрицательность переменных:*

$x_{21} >= 0$
$x_{31} >= 0$
$x_{12} >= 0$
$x_{22} >= 0$
$x_{13} >= 0$
$x_{33} >= 0$

In [12]:
from cvxopt import matrix, solvers

c = matrix([-1 for _ in range(6)], tc='d')
G = matrix([[0.005, 0.004, 0.003, 0.009, 0.003, 0.005],
            [5, 8, 0, 0, 0, 0],
            [0, 0, 20, 8, 0, 0],
            [0, 0, 0, 0, 13, 9],
            [-1, 0, 0, 0, 0, 0],
            [0, -1, 0, 0, 0, 0],
            [0, 0, -1, 0, 0, 0],
            [0, 0, 0, -1, 0, 0],
            [0, 0, 0, 0, -1, 0],
            [0, 0, 0, 0, 0, -1]], tc='d')

h = matrix([max_y_product[0], 860, 1500, 870, 0, 0, 0, 0, 0, 0], tc='d')
solution = solvers.lp(c, G.T, h, solver='glpk')

print(f'Status: {solution["status"]}')
print(f'Objective: {-solution["primal objective"]}')
print(f'x = \n{solution["x"]}')

max_product = -solution['primal objective']
shadow_price, reduced_cost = solution['z'][:-len(c)], solution['z'][-len(c):]
print(f'Shadow price=\n{shadow_price}\nReduced cost=\n{reduced_cost}')


Status: optimal
Objective: -456.1666666666667
x = 
[ 1.72e+02]
[ 0.00e+00]
[ 0.00e+00]
[ 1.88e+02]
[ 0.00e+00]
[ 9.67e+01]

Shadow price=
[-0.00e+00]
[ 2.00e-01]
[ 1.25e-01]
[ 1.11e-01]

Reduced cost=
[-0.00e+00]
[ 6.00e-01]
[ 1.50e+00]
[-0.00e+00]
[ 4.44e-01]
[-0.00e+00]



### Выводы по решению:

1. Ресурс $Y$, производимый цехами $N1$ и $N2$ не является дефицитным.
2. Полностью загружены все группы оборудования, т.к. временной фонд в ограничениях $y2$, $y3$ и $y4$ является дефицитным ресурсом.

In [22]:
for constraint_index, price in enumerate(shadow_price):
    if price == 0:
        continue
    print(f'Sensitivity analysis for y{constraint_index + 1} constraint')
    dh = np.zeros(len(h))
    dh[constraint_index] = 1
    dh = matrix(dh)
    a = 1
    prev_z = -solution['primal objective']
    
    step = 100
    
    while (True):
        solution_i = solvers.lp(c, G.T, h + dh * a, solver='glpk')
        if solution_i['status'] != 'optimal':
            print("Couldn't find a solution")
            break

        new_z = -solution_i['primal objective']
        delta_z = (new_z - prev_z) / step if a != 1 else new_z - prev_z
        prev_z = new_z
        
        print(f'Increment {a}: objective={new_z}, delta={delta_z}')
        
        if abs(delta_z - price) > 1e-6:
            print(f'Basis changed at increment {a}')
            break
            
        a += step
    print()

Sensitivity analysis for y2 constraint
Increment 1: objective=456.3666666666667, delta=0.19999999999998863
Increment 101: objective=476.3666666666667, delta=0.2
Increment 201: objective=496.3666666666667, delta=0.2
Increment 301: objective=516.3666666666667, delta=0.2
Increment 401: objective=536.3666666666667, delta=0.2
Increment 501: objective=556.3666666666667, delta=0.2
Increment 601: objective=576.3666666666667, delta=0.2
Increment 701: objective=596.3666666666667, delta=0.2
Increment 801: objective=616.3666666666667, delta=0.2
Increment 901: objective=636.3666666666667, delta=0.2
Increment 1001: objective=656.3666666666667, delta=0.2
Increment 1101: objective=676.3666666666667, delta=0.2
Increment 1201: objective=696.3666666666667, delta=0.2
Increment 1301: objective=716.3666666666667, delta=0.2
Increment 1401: objective=736.3666666666667, delta=0.2
Increment 1501: objective=756.3666666666667, delta=0.2
Increment 1601: objective=776.3666666666667, delta=0.2
Increment 1701: object

Increment 26001: objective=5656.366666666667, delta=0.2
Increment 26101: objective=5676.366666666667, delta=0.2
Increment 26201: objective=5696.366666666667, delta=0.2
Increment 26301: objective=5716.366666666667, delta=0.2
Increment 26401: objective=5736.366666666667, delta=0.2
Increment 26501: objective=5756.366666666667, delta=0.2
Increment 26601: objective=5776.366666666667, delta=0.2
Increment 26701: objective=5796.366666666667, delta=0.2
Increment 26801: objective=5816.366666666667, delta=0.2
Increment 26901: objective=5836.366666666667, delta=0.2
Increment 27001: objective=5856.366666666667, delta=0.2
Increment 27101: objective=5876.366666666667, delta=0.2
Increment 27201: objective=5896.366666666667, delta=0.2
Increment 27301: objective=5916.366666666667, delta=0.2
Increment 27401: objective=5936.366666666667, delta=0.2
Increment 27501: objective=5956.366666666667, delta=0.2
Increment 27601: objective=5976.366666666667, delta=0.2
Increment 27701: objective=5996.366666666667, de

Increment 41301: objective=8716.366666666667, delta=0.2
Increment 41401: objective=8736.366666666667, delta=0.2
Increment 41501: objective=8756.366666666667, delta=0.2
Increment 41601: objective=8776.366666666667, delta=0.2
Increment 41701: objective=8796.366666666667, delta=0.2
Increment 41801: objective=8816.366666666667, delta=0.2
Increment 41901: objective=8836.366666666667, delta=0.2
Increment 42001: objective=8856.366666666667, delta=0.2
Increment 42101: objective=8876.366666666667, delta=0.2
Increment 42201: objective=8896.366666666667, delta=0.2
Increment 42301: objective=8916.366666666667, delta=0.2
Increment 42401: objective=8936.366666666667, delta=0.2
Increment 42501: objective=8956.366666666667, delta=0.2
Increment 42601: objective=8976.366666666667, delta=0.2
Increment 42701: objective=8996.366666666667, delta=0.2
Increment 42801: objective=9016.366666666667, delta=0.2
Increment 42901: objective=9036.366666666667, delta=0.2
Increment 43001: objective=9056.366666666667, de

Increment 56001: objective=11656.366666666667, delta=0.2
Increment 56101: objective=11676.366666666667, delta=0.2
Increment 56201: objective=11696.366666666667, delta=0.2
Increment 56301: objective=11716.366666666667, delta=0.2
Increment 56401: objective=11736.366666666667, delta=0.2
Increment 56501: objective=11756.366666666667, delta=0.2
Increment 56601: objective=11776.366666666667, delta=0.2
Increment 56701: objective=11796.366666666667, delta=0.2
Increment 56801: objective=11816.366666666667, delta=0.2
Increment 56901: objective=11836.366666666667, delta=0.2
Increment 57001: objective=11856.366666666667, delta=0.2
Increment 57101: objective=11876.366666666667, delta=0.2
Increment 57201: objective=11896.366666666667, delta=0.2
Increment 57301: objective=11916.366666666667, delta=0.2
Increment 57401: objective=11936.366666666667, delta=0.2
Increment 57501: objective=11956.366666666667, delta=0.2
Increment 57601: objective=11976.366666666667, delta=0.2
Increment 57701: objective=1199

Increment 24801: objective=3556.291666666667, delta=0.125
Increment 24901: objective=3568.791666666667, delta=0.125
Increment 25001: objective=3581.291666666667, delta=0.125
Increment 25101: objective=3593.791666666667, delta=0.125
Increment 25201: objective=3606.291666666667, delta=0.125
Increment 25301: objective=3618.791666666667, delta=0.125
Increment 25401: objective=3631.291666666667, delta=0.125
Increment 25501: objective=3643.791666666667, delta=0.125
Increment 25601: objective=3656.291666666667, delta=0.125
Increment 25701: objective=3668.791666666667, delta=0.125
Increment 25801: objective=3681.291666666667, delta=0.125
Increment 25901: objective=3693.791666666667, delta=0.125
Increment 26001: objective=3706.291666666667, delta=0.125
Increment 26101: objective=3718.791666666667, delta=0.125
Increment 26201: objective=3731.291666666667, delta=0.125
Increment 26301: objective=3743.791666666667, delta=0.125
Increment 26401: objective=3756.291666666667, delta=0.125
Increment 2650

...

Increment 50801: objective=6100.722222222223, delta=0.11111111111111313
Increment 50901: objective=6111.833333333333, delta=0.11111111111110404
Increment 51001: objective=6122.944444444444, delta=0.11111111111111313
Increment 51101: objective=6134.055555555556, delta=0.11111111111111313
Increment 51201: objective=6145.166666666667, delta=0.11111111111111313
Increment 51301: objective=6156.277777777777, delta=0.11111111111110404
Increment 51401: objective=6167.388888888889, delta=0.11111111111111313
Increment 51501: objective=6178.5, delta=0.11111111111111313
Increment 51601: objective=6189.611111111111, delta=0.11111111111111313
Increment 51701: objective=6200.722222222223, delta=0.11111111111111313
Increment 51801: objective=6211.833333333333, delta=0.11111111111110404
Increment 51901: objective=6222.944444444444, delta=0.11111111111111313
Increment 52001: objective=6234.055555555556, delta=0.11111111111111313
Increment 52101: objective=6245.166666666667, delta=0.11111111111111313
Inc

Increment 65301: objective=7711.833333333334, delta=0.11111111111111313
Increment 65401: objective=7722.944444444445, delta=0.11111111111111313
Increment 65501: objective=7734.055555555556, delta=0.11111111111110404
Increment 65601: objective=7745.166666666667, delta=0.11111111111111313
Increment 65701: objective=7756.277777777778, delta=0.11111111111111313
Increment 65801: objective=7767.38888888889, delta=0.11111111111111313
Increment 65901: objective=7778.5, delta=0.11111111111110404
Increment 66001: objective=7789.611111111111, delta=0.11111111111111313
Increment 66101: objective=7800.722222222223, delta=0.11111111111111313
Increment 66201: objective=7811.833333333334, delta=0.11111111111111313
Increment 66301: objective=7822.944444444444, delta=0.11111111111110404
Increment 66401: objective=7834.055555555556, delta=0.11111111111111313
Increment 66501: objective=7845.166666666667, delta=0.11111111111111313
Increment 66601: objective=7856.277777777778, delta=0.11111111111111313
Incr

Increment 79701: objective=9311.833333333334, delta=0.11111111111111313
Increment 79801: objective=9322.944444444445, delta=0.11111111111111313
Increment 79901: objective=9334.055555555555, delta=0.11111111111109494
Increment 80001: objective=9345.166666666666, delta=0.11111111111111313
Increment 80101: objective=9356.277777777777, delta=0.11111111111111313
Increment 80201: objective=9367.388888888889, delta=0.11111111111111313
Increment 80301: objective=9378.5, delta=0.11111111111111313
Increment 80401: objective=9389.611111111111, delta=0.11111111111111313
Increment 80501: objective=9400.722222222223, delta=0.11111111111111313
Increment 80601: objective=9411.833333333334, delta=0.11111111111111313
Increment 80701: objective=9422.944444444445, delta=0.11111111111111313
Increment 80801: objective=9434.055555555555, delta=0.11111111111109494
Increment 80901: objective=9445.166666666666, delta=0.11111111111111313
Increment 81001: objective=9456.277777777777, delta=0.11111111111111313
Inc

Increment 94001: objective=10900.722222222223, delta=0.11111111111111313
Increment 94101: objective=10911.833333333334, delta=0.11111111111111313
Increment 94201: objective=10922.944444444445, delta=0.11111111111111313
Increment 94301: objective=10934.055555555555, delta=0.11111111111109494
Increment 94401: objective=10945.166666666666, delta=0.11111111111111313
Increment 94501: objective=10956.277777777777, delta=0.11111111111111313
Increment 94601: objective=10967.388888888889, delta=0.11111111111111313
Increment 94701: objective=10978.5, delta=0.11111111111111313
Increment 94801: objective=10989.611111111111, delta=0.11111111111111313
Increment 94901: objective=11000.722222222223, delta=0.11111111111111313
Increment 95001: objective=11011.833333333334, delta=0.11111111111111313
Increment 95101: objective=11022.944444444445, delta=0.11111111111111313
Increment 95201: objective=11034.055555555555, delta=0.11111111111109494
Increment 95301: objective=11045.166666666666, delta=0.1111111

Increment 109201: objective=12589.611111111111, delta=0.11111111111111313
Increment 109301: objective=12600.722222222223, delta=0.11111111111111313
Increment 109401: objective=12611.833333333334, delta=0.11111111111111313
Increment 109501: objective=12622.944444444445, delta=0.11111111111111313
Increment 109601: objective=12634.055555555555, delta=0.11111111111109494
Increment 109701: objective=12645.166666666666, delta=0.11111111111111313
Increment 109801: objective=12656.144193165343, delta=0.10977526498676525
Basis changed at increment 109801



## Задача 3.

$x_{ij}$, количество перевезённого продукта i в пункт j


$f(x) = 5.1x_{11} + 7.4x_{12} + 7.6x_{13} + 5.3x_{14} + 3.0x_{15} + 5.6x_{21} + 7.4x_{22} + 4.0x_{23} + 7.9x_{24} + 6.6x_{25} + 2.2x_{31} + 4.3x_{32} + 5.7x_{33} + 5.8x_{34} + 6.6x_{35} + 5.1x_{41} + 5.3x_{42} + 3.3x_{43} + 6.7x_{44} + 6.8x_{45}$  

$x_{11} + x_{12} + x_{13} + x_{14} + x_{15} <= 0.67 \cdot prev$  
$x_{21} + x_{22} + x_{23} + x_{24} + x_{25} <= 0.67 \cdot 4400$  
$x_{31} + x_{32} + x_{33} + x_{34} + x_{35} <= 0.67 \cdot 5900$  
$x_{41} + x_{42} + x_{43} + x_{44} + x_{45} <= 0.67 \cdot 4200$  
$x_{51} + x_{52} + x_{53} + x_{54} + x_{55} <= 5579$  
$x_{11} + x_{21} + x_{31} + x_{41} + x_{51} = 1900$  
$x_{12} + x_{22} + x_{32} + x_{42} + x_{52} = 3200$  
$x_{13} + x_{23} + x_{33} + x_{43} + x_{53} = 2900$  
$x_{14} + x_{24} + x_{34} + x_{44} + x_{54} = 4100$  
$x_{15} + x_{25} + x_{35} + x_{45} + x_{55} = 3500$  
$x_{ij} >= 0$


In [5]:
import numpy as np

manufactured = [0.67 * i for i in [max_product, 4400, 5900, 4200]]
required = [1900, 3200, 2900, 4100, 3500]
delta = sum(required) - sum(manufactured)
print(f"Manufactured = {sum(manufactured)}, required = {sum(required)}, delta = {sum(required) - sum(manufactured)}")


c = matrix([5.1, 7.4, 7.6, 5.3, 0, 3.0, 5.6, 7.4, 4.0, 0, 7.9, 6.6, 2.2, 4.3, 0, 5.7, 5.8, 6.6, 5.1, 0, 5.3, 3.3, 6.7, 6.8, 0], tc='d')
G = matrix(-np.eye(25), tc='d')
h = matrix(np.zeros(25), tc='d')

A = matrix([[1, 1, 1, 1, 1,  0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0,  1, 1, 1, 1, 1,  0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  1, 1, 1, 1, 1,  0, 0, 0, 0, 0,  0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  1, 1, 1, 1, 1,  0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  0, 0, 0, 0, 0,  1, 1, 1, 1, 1],
     
            [1, 0, 0, 0, 0,  1, 0, 0, 0, 0,  1, 0, 0, 0, 0,  1, 0, 0, 0, 0,  1, 0, 0, 0, 0],
            [0, 1, 0, 0, 0,  0, 1, 0, 0, 0,  0, 1, 0, 0, 0,  0, 1, 0, 0, 0,  0, 1, 0, 0, 0],
            [0, 0, 1, 0, 0,  0, 0, 1, 0, 0,  0, 0, 1, 0, 0,  0, 0, 1, 0, 0,  0, 0, 1, 0, 0],
            [0, 0, 0, 1, 0,  0, 0, 0, 1, 0,  0, 0, 0, 1, 0,  0, 0, 0, 1, 0,  0, 0, 0, 1, 0],
            [0, 0, 0, 0, 1,  0, 0, 0, 0, 1,  0, 0, 0, 0, 1,  0, 0, 0, 0, 1,  0, 0, 0, 0, 1]], tc='d')

b = matrix([0.67 * max_product,
            0.67 * 4400,
            0.67 * 5900,
            0.67 * 4200,
            delta,
     
            1900,
            3200,
            2900,
            4100,
            3500], tc='d')

transport_solution = solvers.lp(c, G.T, h, A.T, b, solver='glpk')
print(f'Minimal cost = {transport_solution["status"]}')
print(f'Plan =\n {transport_solution["x"]}')
print(transport_solution)

Manufactured = 10020.631666666668, required = 15600, delta = 5579.368333333332
Minimal cost = optimal
Plan =
 [ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 3.06e+02]
[ 1.90e+03]
[ 0.00e+00]
[ 0.00e+00]
[ 1.05e+03]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 2.90e+03]
[ 1.05e+03]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 2.00e+03]
[ 8.15e+02]
[ 0.00e+00]
[ 3.20e+03]
[ 0.00e+00]
[ 0.00e+00]
[ 2.38e+03]

{'status': 'optimal', 'x': <25x1 matrix, tc='d'>, 's': <25x1 matrix, tc='d'>, 'y': <10x1 matrix, tc='d'>, 'z': <25x1 matrix, tc='d'>, 'primal objective': 41554.8, 'dual objective': 41554.8, 'gap': 0.0, 'relative gap': 0.0, 'primal infeasibility': 8.48926977216425e-17, 'dual infeasibility': 1.0272173429066128e-16, 'primal slack': 0.0, 'dual slack': -0.0, 'residual as primal infeasibility certificate': None, 'residual as dual infeasibility certificate': None}
